In [4]:
import numpy as np

y_predicted = np.array([1,1,0,0,1])
y_true = np.array([0.3,0.7,1,0,0.5])

In [16]:
# Actual definition of np.mean & np.abs fn in numpy :

def MeanAbsoluteErr(y_true,y_predicted):
    total_err = 0
    for yt,yp in zip(y_true,y_predicted):
        total_err += abs(yt - yp)
    print("Total err:", total_err)
    
    mae = total_err / len(y_true)
    print("MAE",mae)
    return mae

In [17]:
MeanAbsoluteErr(y_true,y_predicted)

Total err: 2.5
MAE 0.5


np.float64(0.5)

In [15]:
np.mean(np.abs(y_predicted - y_true))

np.float64(0.5)